# An example of mastrplan creation

In [1]:
import pandas as pd
import networkx as nx
import geopandas as gpd

from masterplan_tools.City_model.city_model import CityModel
from masterplan_tools.Provision_getter.provision_getter import ProvisionModel
from masterplan_tools.Balancer.balancer import MasterPlan
from masterplan_tools.Data_getter.data_getter import DataGetter

c:\masterplanning\env\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


#### City model creation

In [2]:
# To assess services provision they must be specified as a list of strings
# Service names must be the same as in your data

service_types = ["schools", "kindergartens"]

In [3]:
# path to data
path = "../masterplanning/mp_data/"

# load data required for blocks creation
city_geometry = gpd.read_parquet(path + 'city_geometry.parquet')
water_geometry = gpd.read_parquet(path + 'water_geometry.parquet')
roads_geometry = gpd.read_parquet(path + 'roads_geometry.parquet')
railways_geometry = gpd.read_parquet(path + 'railways_geometry.parquet')
nature_geometry_boundaries = gpd.read_parquet(path + 'nature_geometry_boundaries.parquet')

# load data required for service graphs creation
schools = gpd.read_parquet(path + 'schools.parquet')
kindergartens = gpd.read_parquet(path + 'kindergartens.parquet')
# accessibility_matrix = pd.read_pickle(path + "accessibility_matrix.pkl")

# load data required for 
buildings = gpd.read_parquet(path + 'buildings.parquet')
greenings = gpd.read_parquet(path + 'greenings.parquet')
parkings = gpd.read_parquet(path + 'parkings.parquet')
transport_graph = nx.read_graphml(path + 'new_graph.graphml')

In [4]:
# services should be specified as a dictionary
services = {
    'schools': schools,
    'kindergartens': kindergartens,
    # 'recreational_areas': добавить рекреационные зоны
}

In [5]:
# City data model creation
city_model = CityModel(
    services=services,
    city_geometry=city_geometry,
    water_geometry=water_geometry,
    roads_geometry=roads_geometry,
    railways_geometry=railways_geometry,
    nature_geometry_boundaries=nature_geometry_boundaries,
    # accessibility_matrix=accessibility_matrix,
    transport_graph=transport_graph,
    buildings=buildings,
    greenings=greenings,
    parkings=parkings
)


2023-05-23 10:46:08.353 | INFO     | masterplan_tools.Blocks_getter.blocks_getter:_cut_railways:133 - Starting: cutting railways geometries
2023-05-23 10:46:08.378 | INFO     | masterplan_tools.Blocks_getter.blocks_getter:_cut_railways:140 - Finished: cutting railways geometries
2023-05-23 10:46:08.379 | INFO     | masterplan_tools.Blocks_getter.blocks_getter:_cut_roads:151 - Starting: cutting roads geometries
2023-05-23 10:46:08.808 | INFO     | masterplan_tools.Blocks_getter.blocks_getter:_cut_roads:158 - Finished: cutting roads geometries
2023-05-23 10:46:08.808 | INFO     | masterplan_tools.Blocks_getter.blocks_getter:_cut_nature:171 - Starting: cutting nature geometries
2023-05-23 10:46:08.931 | INFO     | masterplan_tools.Blocks_getter.blocks_getter:_cut_nature:178 - Finished: cutting nature geometries
2023-05-23 10:46:08.932 | INFO     | masterplan_tools.Blocks_getter.blocks_getter:_fill_deadends:91 - Starting: filling deadends
2023-05-23 10:46:09.032 | INFO     | masterplan_too

In [6]:
city_model.services_graph.nodes(data=True)

NodeDataView({3: {'population': 3946.0, 'is_living': True, 'is_schools_service': 1, 'schools_capacity': 1200, 'provision_schools': 0, 'id_schools': 0, 'population_prov_schools': 0, 'population_unprov_schools': 3946.0, 'is_kindergartens_service': 0, 'provision_kindergartens': 0, 'id_kindergartens': 0, 'population_prov_kindergartens': 0, 'population_unprov_kindergartens': 3946.0}, 0: {'population': 0.0, 'is_living': False, 'is_schools_service': 0, 'provision_schools': 0, 'id_schools': 0, 'is_kindergartens_service': 0, 'provision_kindergartens': 0, 'id_kindergartens': 0}, 1: {'population': 0.0, 'is_living': False, 'is_schools_service': 0, 'provision_schools': 0, 'id_schools': 0, 'is_kindergartens_service': 0, 'provision_kindergartens': 0, 'id_kindergartens': 0}, 2: {'population': 3375.0, 'is_living': True, 'is_schools_service': 0, 'provision_schools': 0, 'id_schools': 0, 'population_prov_schools': 0, 'population_unprov_schools': 3375.0, 'is_kindergartens_service': 0, 'provision_kindergart

#### Calculating the provision of school and kindergarten services in blocks

In [7]:
for service_type in services.keys():    
    model = ProvisionModel(
        city_model=city_model,
        service_name=service_type
        )

    if service_type == 'schools':
        school = model.run()
        
    elif service_type == 'kindergartens':
        kindergarten = model.run()


KeyError: 'schools'

#### Results of calculating the provision of the kindergarten service for a single block

In [7]:
kindergarten.iloc[10]

index                                                                             10
id                                                                                10
geometry                           POLYGON ((343053.994039182 6648458.059121506, ...
provision_kindergartens                                                          100
id_kindergartens                                                                  36
population_prov_kindergartens                                                  20531
population_unprov_kindergartens                                                    0
population                                                                     20531
Name: 10, dtype: object

#### Results of calculating the provision of the school service for a single block

In [8]:
school.iloc[10]

index                                                                       10
id                                                                          10
geometry                     POLYGON ((343053.994039182 6648458.059121506, ...
provision_schools                                                          100
id_schools                                                                  37
population_prov_schools                                                  20531
population_unprov_schools                                                    0
population                                                               20531
Name: 10, dtype: object

#### Selected area of the city to which the new parameters will be modelled

In [9]:
polygon = gpd.read_file('../masterplanning/masterplan_tools/output_data/polygon.geojson')
polygon.explore()

DriverError: ../masterplanning/masterplan_tools/output_data/polygon.geojson: No such file or directory

#### Modelling of new parameters for the selected area with maximisation of the number of inhabitants

In [ ]:
block = DataGetter().balance_data(gdf=city_model.blocks_aggregated_info, polygon=polygon, school=school, kindergarten=kindergarten, greening=greening)
mp = MasterPlan(area=block['area'],
                current_living_area=block['current_living_area'],
                current_industrial_area=block['current_industrial_area'],
                current_population=block['current_population'],
                current_green_area=block['current_green_area'],
                current_unprov_schoolkids=block['population_unprov_schools'],
                current_unprov_kids=block['population_unprov_kindergartens'],
                current_unprov_green_population=block['population_unprov_recreational_areas'])

updated_block = mp.optimal_solution_indicators()


#### Modelled block transformation requirements

In [ ]:
updated_block['block_id'] = 15
updated_block

NameError: name 'updated_block' is not defined

#### Making changes based on requirements in the data model

In [ ]:
service_graphs = {}
for service_type in city_model.service_types:
   service_graphs[service_type] = DataGetter().prepare_graph(
                    blocks=city_model.city_blocks,
                    service_type=service_type,
                    buildings=city_model.buildings,
                    service_gdf=city_model.services_gdfs[service_type],
                    updated_block_info=updated_block,
                    accessibility_matrix=city_model.accessibility_matrix,
                )

In [ ]:
city_model.services_graphs = service_graphs

#### Recalculating the provision of services to blocks

In [ ]:
for service_type in city_model.service_types:    
    model = ProvisionModel(
    service_name=service_type,
    city_model=city_model
    )

    if service_type == 'schools':
        school = model.run()
        
    elif service_type == 'kindergartens':
        kindergarten = model.run()

количество кварталов c сервисом schools: 29
количество жилых кварталов: 1273
количество кварталов всего: 1609
количество кварталов c ошибкой: 0
{'population': 18454.0, 'is_living': True, 'is_schools_service': 1, 'schools_capacity': 3000, 'provision_schools': 0, 'id_schools': 0, 'population_prov_schools': 0, 'population_unprov_schools': 18454.0}
количество кварталов c сервисом kindergartens: 11
количество жилых кварталов: 797
количество кварталов всего: 900
количество кварталов c ошибкой: 0
{'population': 18454.0, 'is_living': True, 'is_kindergartens_service': 1, 'kindergartens_capacity': 743, 'provision_kindergartens': 0, 'id_kindergartens': 0, 'population_prov_kindergartens': 0, 'population_unprov_kindergartens': 18454.0}


#### Modified data on the availability of specified services

In [ ]:
school.iloc[15]

index                                                                     6013
id                                                                        6013
geometry                     POLYGON ((365716.43716688757 6624359.046022324...
provision_schools                                                          100
id_schools                                                                6013
population_prov_schools                                                  18454
population_unprov_schools                                                    0
population                                                               18454
Name: 6013, dtype: object

In [ ]:
kindergarten.iloc[15]

index                                                                           6013
id                                                                              6013
geometry                           POLYGON ((365716.43716688757 6624359.046022324...
provision_kindergartens                                                           66
id_kindergartens                                                                6013
population_prov_kindergartens                                                  12180
population_unprov_kindergartens                                                 6273
population                                                                     18454
Name: 6013, dtype: object